In [16]:
# Imports 
import numpy as np
import gdsfactory as gf
from functools import partial
import math 
from gdsfactory.read.from_np import from_image
from gdsfactory.path import Path, extrude

#gf.CONF.cell_layout_cache = False
#c.plot()

# PARAMETERS

In [17]:
# Parameters definition

layer_metal = (4,0)
layer_box = (100,0)

die_w = 10000
die_L = 5000

dicing_edge = 125 

"""
layer_shallow = (1,0)
layer_deep = (2,0)
layer_prot_deep = (11,0)
layer_prot_shallow = (21,0)


num_per_side = 37
spacing = 127
die_side = 'left'
offset_x = dicing_edge/2
offset_y = dicing_edge

w_wg_gen = 1.0+0.2
w_wg_w1 = 2.5+0.2
w_wg_w2 = 4.0+0.2

w_wg_md_gen = 1.0+0.2
w_wg_md_w1 = 2.5+0.2
w_wg_md_w2 = 4.0+0.2

w_wg_shal_gen = 1.0+0.2
w_wg_shal_w1 = 2.5+0.2
w_wg_shal_w2 = 4.0+0.2

w_slab = 15.0
w_prot_deep = 15.0

w_taper_io = 0.75+0.2

h_wg_gen = 0.3
h_wg_mini = 0.15

w_metal = 3.5
"""

"\nlayer_shallow = (1,0)\nlayer_deep = (2,0)\nlayer_prot_deep = (11,0)\nlayer_prot_shallow = (21,0)\n\n\nnum_per_side = 37\nspacing = 127\ndie_side = 'left'\noffset_x = dicing_edge/2\noffset_y = dicing_edge\n\nw_wg_gen = 1.0+0.2\nw_wg_w1 = 2.5+0.2\nw_wg_w2 = 4.0+0.2\n\nw_wg_md_gen = 1.0+0.2\nw_wg_md_w1 = 2.5+0.2\nw_wg_md_w2 = 4.0+0.2\n\nw_wg_shal_gen = 1.0+0.2\nw_wg_shal_w1 = 2.5+0.2\nw_wg_shal_w2 = 4.0+0.2\n\nw_slab = 15.0\nw_prot_deep = 15.0\n\nw_taper_io = 0.75+0.2\n\nh_wg_gen = 0.3\nh_wg_mini = 0.15\n\nw_metal = 3.5\n"

# PDK CELLS

In [18]:
gf.clear_cache()

@gf.cell
def die(dieL = 5000, dieW = 10000, border = 250, layer_box = (101,0)):
    box = gf.Component()
    obox = box.add_ref(gf.components.rectangle(size=(dieW,dieL),layer=layer_box))
    ibox = box.add_ref(gf.components.rectangle(size=(dieW-border*2,dieL-border*2),layer=layer_box)).dmovex(border).dmovey(border)
    box = gf.boolean(A=obox, B=ibox, operation="A-B", layer=layer_box)
    box.add_port(name="block@org", center=[border,border], width=1, orientation=0, layer=layer_box)
    return box

In [19]:
wafer = gf.Component(name='w')

c1 = gf.Component(name='c1')

box7 = c1.add_ref(die(dieW = die_w, dieL = die_L, border = dicing_edge, layer_box=layer_box ))

# RESISTENCIAS

In [20]:
import gdsfactory as gf
from gdsfactory.component import Component
from gdsfactory.components import rectangle, taper  # función que genera un rectángulo de tamaño (x, y) en una capa específica. Genera un taper.


In [21]:
gf.clear_cache()

@gf.cell
def heater_resistor(layer = (4,0),
                    pad_xw = 110,
                    pad_yw = 110,
                    rad_pad = 50,
                    taper_len = 100,
                    l_res = 1300,
                    w_res = 10):
    
    c = gf.Component()
    xs_pad = gf.cross_section.heater_metal(width=pad_xw,radius=rad_pad, layer=layer)
    xs_res = gf.cross_section.heater_metal(width=w_res,radius=rad_pad, layer=layer)

    res_c = gf.components.straight(length=l_res, cross_section=xs_res)    
    taper_c = gf.components.taper_cross_section_parabolic(cross_section1=xs_res, cross_section2=xs_pad, length=taper_len, npoints=201)
    bend_c = gf.components.bend_euler(radius=rad_pad, cross_section=xs_res)
    pad_c = gf.components.pad(size=(pad_xw, pad_yw), layer=layer)
    # taper_c.pprint_ports()
    # pad_c.pprint_ports()
    res_r = c.add_ref(res_c)
    taper_r1 = c.add_ref(taper_c)
    taper_r2 = c.add_ref(taper_c)
    bend_r1 = c.add_ref(bend_c).dmirror_y()
    bend_r2 = c.add_ref(bend_c)
    pad_r1 = c.add_ref(pad_c)
    pad_r2 = c.add_ref(pad_c)

    taper_r1.connect("e2", pad_r1.ports["e2"])
    bend_r1.connect("e1", taper_r1.ports["e1"])
    res_r.connect("e1", bend_r1.ports["e2"])
    bend_r2.connect("e2", res_r.ports["e2"])
    taper_r2.connect("e1", bend_r2.ports["e1"])
    pad_r2.connect("e1", taper_r2.ports["e2"])
    c.dmovex(-rad_pad).dmovey(-(0.5*pad_yw+taper_len+rad_pad))
    return c

@gf.cell
def heater_ref(layer = (4,0),
                    pad_xw = 110,
                    pad_yw = 110,
                    rad_pad = 50,
                    taper_len = 100,
                    l_res = 0.1,
                    w_res = 10):
    
    c = gf.Component()
    xs_pad = gf.cross_section.heater_metal(width=pad_xw,radius=rad_pad, layer=layer)
    xs_res = gf.cross_section.heater_metal(width=w_res,radius=rad_pad, layer=layer)

    res_c = gf.components.straight(length=l_res, cross_section=xs_res)    
    taper_c = gf.components.taper_cross_section_parabolic(cross_section1=xs_res, cross_section2=xs_pad, length=taper_len, npoints=201)
    bend_c = gf.components.bend_euler(radius=rad_pad, cross_section=xs_res)
    pad_c = gf.components.pad(size=(pad_xw, pad_yw), layer=layer)
    # taper_c.pprint_ports()
    # pad_c.pprint_ports()
    res_r = c.add_ref(res_c)
    taper_r1 = c.add_ref(taper_c)
    taper_r2 = c.add_ref(taper_c)
    bend_r1 = c.add_ref(bend_c).dmirror_y()
    bend_r2 = c.add_ref(bend_c).dmirror_y()
    pad_r1 = c.add_ref(pad_c)
    pad_r2 = c.add_ref(pad_c)

    taper_r1.connect("e2", pad_r1.ports["e2"])
    bend_r1.connect("e1", taper_r1.ports["e1"])
    res_r.connect("e1", bend_r1.ports["e2"])
    bend_r2.connect("e2", res_r.ports["e2"])
    taper_r2.connect("e1", bend_r2.ports["e1"])
    pad_r2.connect("e1", taper_r2.ports["e2"])
    c.dmovex(-rad_pad).dmovey(-(0.5*pad_yw+taper_len+rad_pad))
    return c

layer = (5,0)
pad_xw = 110
pad_yw = 110
rad_pad = 20
taper_len = 100
l_res = 0.1
w_res = 10

c = gf.Component()
c.add_ref(heater_ref())
#c.add_ref(heater_resistor(w_res=20)).dmirror_y().dmovey(100)

c.show()

2025-11-25 11:25:56.714 | WARNING  | kfactory.kcell:show:4100 - klive didn't send data, closing


In [22]:
layer_metal = (5,0)
pad_xw = 110
pad_yw = 110
rad_pad = 20
taper_len = 70
l_res = 1600
#w_res_1 = 10
w_res_1 = 10
w_res_2 = 20

base = gf.read.import_gds('L6_CNM-PHYSIS_RUBEN_RIGHTside_2023-07-11_CNMPDK_FINALISIMO.gds')

c = gf.Component()
c.add_ref(base)
#c.show()

Unnamed_58: ports [], KCell(name=CNM-PHYSIS, ports=[], pins=[], instances=[], locked=False, kcl=DEFAULT)

In [23]:
# TODO: Change all the coordinates ussing the new cell. 
# heater_resistor. Now the (0,0) of the component is located 
# at the beggining of the heater.

offset_y = 1.7
w_wg = 1.1

res = gf.Component()

res1 = gf.Component()
res2 = gf.Component()
res3 = gf.Component()


deltaRes1 = -482.1+2*(offset_y)+w_wg+w_res_1
deltaRes2 = -482.1+2*(offset_y)+w_wg+w_res_2

m2_top = res1.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_1
))
#m2_top.dmovex().dmovey()

m2_bot = res1.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_1
))
#m2_bot.dmirror_y().dmovex(0).dmovey(-481.2+3.2+w_res_1)
m2_bot.dmirror_y().dmovex(0).dmovey(deltaRes1)

m3_top = res2.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_1
))
m3_top.dmovex(3210).dmovey(0)

m3_bot = res2.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_1
))
m3_bot.dmirror_y().dmovex(3210).dmovey(deltaRes1)

m4_top = res3.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_2
))
m4_top.dmovex(2*3210).dmovey(-0.5*(w_res_2-w_res_1))

m4_bot = res3.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_2
))
m4_bot.dmirror_y().dmovex(2*3210).dmovey(-0.5*(w_res_2-w_res_1)+deltaRes2)

m1_top = res1.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_1
))
m1_top.dmovex(0).dmovey(586.1)

m1_bot = res1.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_1
))
#m1_bot.dmirror_y().dmovex(0).dmovey(586.1-481.2+3.2+w_res_1)
m1_bot.dmirror_y().dmovex(0).dmovey(586.1+deltaRes1)

m5_top = res3.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_2
))
m5_top.dmovex(2*3210).dmovey(-586.1-0.5*(w_res_2-w_res_1))

m5_bot = res3.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_2
))
m5_bot.dmirror_y().dmovex(2*3210).dmovey(-586.1-0.5*(w_res_2-w_res_1)+deltaRes2)

msh2_top = res2.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_1
))
msh2_top.dmovex(3210+234.2).dmovey(2151)

msh2_bot = res2.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_1
))
msh2_bot.dmirror_y().dmovex(3210+234.2).dmovey(2151+deltaRes1)

msh6_top = res2.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_1
))
msh6_top.dmovex(3210+234.2).dmovey(-1973.9)

msh6_bot = res2.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_1
))
msh6_bot.dmirror_y().dmovex(3210+234.2).dmovey(-1973.9+deltaRes1)

msh3_top = res2.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_2
))
msh3_top.dmovex(3210+234.2).dmovey(-0.5*(w_res_2-w_res_1)+1074.1)

msh3_bot = res2.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_2
))
msh3_bot.dmirror_y().dmovex(3210+234.2).dmovey(-0.5*(w_res_2-w_res_1)+1074.1+deltaRes2)

msh5_top = res2.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_2
))
msh5_top.dmovex(3210+234.2).dmovey(-0.5*(w_res_2-w_res_1)-897)

msh5_bot = res2.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_2
))
msh5_bot.dmirror_y().dmovex(3210+234.2).dmovey(-0.5*(w_res_2-w_res_1)-897+deltaRes2)

ref_msh2_1 = res2.add_ref(heater_ref(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=0.1,
    w_res=w_res_1
))
ref_msh2_1.dmovex(3210+234.2+200).dmovey(2151-240)
ref_msh2_2 = res2.add_ref(heater_ref(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=0.1,
    w_res=w_res_1
))
ref_msh2_2.dmovex(3210+234.2+800).dmovey(2151-240)
ref_msh2_3 = res2.add_ref(heater_ref(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=0.1,
    w_res=w_res_1
))
ref_msh2_3.dmovex(3210+234.2+1400).dmovey(2151-240)
ref_msh3_1 = res2.add_ref(heater_ref(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=0.1,
    w_res=w_res_2
))
ref_msh3_1.dmovex(3210+234.2+200).dmovey(-0.5*(w_res_2-w_res_1)+1074.1-229)
ref_msh3_2 = res2.add_ref(heater_ref(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=0.1,
    w_res=w_res_2
))
ref_msh3_2.dmovex(3210+234.2+800).dmovey(-0.5*(w_res_2-w_res_1)+1074.1-229)
ref_msh3_3 = res2.add_ref(heater_ref(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=0.1,
    w_res=w_res_2
))
ref_msh3_3.dmovex(3210+234.2+1400).dmovey(-0.5*(w_res_2-w_res_1)+1074.1-229)

ref_msh6_1 = res2.add_ref(heater_ref(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=0.1,
    w_res=w_res_1
))
ref_msh6_1.dmovex(3210+234.2+200).dmovey(-1973.9-240)
ref_msh6_2 = res2.add_ref(heater_ref(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=0.1,
    w_res=w_res_1
))
ref_msh6_2.dmovex(3210+234.2+800).dmovey(-1973.9-240)
ref_msh6_3 = res2.add_ref(heater_ref(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=0.1,
    w_res=w_res_1
))
ref_msh6_3.dmovex(3210+234.2+1400).dmovey(-1973.9-240)


ref_msh5_1 = res2.add_ref(heater_ref(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=0.1,
    w_res=w_res_2
))
ref_msh5_1.dmovex(3210+234.2+200).dmovey(-0.5*(w_res_2-w_res_1)-897.9-229)
ref_msh5_2 = res2.add_ref(heater_ref(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=0.1,
    w_res=w_res_2
))
ref_msh5_2.dmovex(3210+234.2+800).dmovey(-0.5*(w_res_2-w_res_1)-897-229)
ref_msh5_3 = res2.add_ref(heater_ref(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=0.1,
    w_res=w_res_2
))
ref_msh5_3.dmovex(3210+234.2+1400).dmovey(-0.5*(w_res_2-w_res_1)-897-229)

res.show()

"""
pad_bet_res = 250
x_loc = 8570
ref_33 = c1.add_ref(c_ref_33)
ref_33.dmovey(3400).dmovex(x_loc)
ref_33_2 = c1.add_ref(c_ref_33)
ref_33_2.dmovey(ref_33.dy+pad_bet_res).dmovex(x_loc)

ref_37 = c1.add_ref(c_ref_37)
ref_37.dmovey(ref_33_2.dy+pad_bet_res).dmovex(x_loc)
ref_37_2 = c1.add_ref(c_ref_37)
ref_37_2.dmovey(ref_37.dy+pad_bet_res).dmovex(x_loc)

ref_50 = c1.add_ref(c_ref_50)
ref_50.dmovey(ref_37_2.dy+pad_bet_res).dmovex(x_loc)
ref_50_2 = c1.add_ref(c_ref_50)
ref_50_2.dmovey(ref_50.dy+pad_bet_res).dmovex(x_loc)


#res_msh1_up.plot()"
"""

2025-11-25 11:26:01.879 | WARNING  | kfactory.kcell:show:4100 - klive didn't send data, closing


'\npad_bet_res = 250\nx_loc = 8570\nref_33 = c1.add_ref(c_ref_33)\nref_33.dmovey(3400).dmovex(x_loc)\nref_33_2 = c1.add_ref(c_ref_33)\nref_33_2.dmovey(ref_33.dy+pad_bet_res).dmovex(x_loc)\n\nref_37 = c1.add_ref(c_ref_37)\nref_37.dmovey(ref_33_2.dy+pad_bet_res).dmovex(x_loc)\nref_37_2 = c1.add_ref(c_ref_37)\nref_37_2.dmovey(ref_37.dy+pad_bet_res).dmovex(x_loc)\n\nref_50 = c1.add_ref(c_ref_50)\nref_50.dmovey(ref_37_2.dy+pad_bet_res).dmovex(x_loc)\nref_50_2 = c1.add_ref(c_ref_50)\nref_50_2.dmovey(ref_50.dy+pad_bet_res).dmovex(x_loc)\n\n\n#res_msh1_up.plot()"\n'

In [24]:
# res.write_gds('Resistencias.gds')

# res1.write_gds('Resistencias_c1.gds')
# res2.dmovex(-3210)
# res2.write_gds('Resistencias_c2.gds')
# res3.dmovex(-2*3210)
# res3.write_gds('Resistencias_c3.gds')


In [31]:
# Heaters de referencia
refs = gf.Component()
ref_w1_0 = refs.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_1
))
ref_w1_1 = refs.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_1
))
ref_w1_1.dmovey(400)
ref_w2_0 = refs.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_2
))
ref_w2_0.dmovey(2*400)

ref_w2_1 = refs.add_ref(heater_resistor(
    layer=layer_metal,
    pad_xw=pad_xw,
    pad_yw=pad_yw,
    rad_pad=rad_pad,
    taper_len=taper_len,
    l_res=l_res,
    w_res=w_res_2
))
ref_w2_1.dmovey(3*400)

refs.show()
refs.write_gds('Referencias.gds')

2025-11-25 11:30:24.967 | WARNING  | kfactory.kcell:show:4100 - klive didn't send data, closing


WindowsPath('Referencias.gds')

In [ ]:
#offset_y = 1.6

# c.add_ref(res).dmovex(1071).dmovey(2660.45-offset_y-0.5*w_res_1)

# c.show()
# c.write_gds('Resistencias_Superpuestas.gds')

2025-11-25 11:26:11.993 | WARNING  | kfactory.kcell:show:4100 - klive didn't send data, closing


WindowsPath('Resistencias_Superpuestas.gds')

In [27]:
## Oblea 

# #wafer.add_ref(c)
# wafer.add_ref(res)
# wafer.show()

# wafer.write_gds("Resistencias_Ruben.gds")